# Using Embedding to Study the Latent Space of the Dating Profiles

## Data Preprocessing

In [1]:
import os
import pandas as pd
import numpy as np

# load cleaned_okc.csv
df = pd.read_csv('../cleaned_okc.csv')

# preview the data
df.head()

,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,"Favorite books, movies, show, music, and food",The six things I could never do without,I spend a lot of time thinking about,On a typical Friday night I am,The most private thing I am willing to admit,You should message me if…,merged_profile,cleaned_religion,seriousness_degree,belief
0,22,single,m,straight,a little extra,strictly anything,socially,never,working on college/university,"asian, white",...,"books: absurdistan, the republic, of mice and ...",food. water. cell phone. shelter.,duality and humorous things,trying to find someone to hang out with. i am ...,i am new to california and looking for someone...,you want to be swept off your feet! you are ti...,about me: i would love to think that i was so...,agnosticism,4.0,agnosticism
1,35,single,m,straight,average,mostly other,often,sometimes,working on space camp,white,...,i am die hard christopher moore fan. i don't r...,delicious porkness in all of its glories. my b...,NaN,NaN,i am very open and will share just about anyth...,NaN,i am a chef: this is what that means. 1. i am ...,agnosticism,2.0,agnosticism
2,29,single,m,straight,average,mostly anything,socially,NaN,graduated from college/university,white,...,"books: to kill a mockingbird, lord of the ring...","like everyone else, i love my friends and fami...",what my contribution to the world is going to ...,out with my friends!,i cried on my first day at school because a bi...,you're awesome.,"i'm an australian living in san francisco, but...",atheism,0.0,atheism
3,31,single,f,straight,average,mostly anything,socially,never,graduated from college/university,white,...,"i like: alphabetized lists, aquariums, autobio...","friends, family, notebook/pen, books, music, t...",things that amuse and inspire me,out and about or relaxing at home with a good ...,NaN,NaN,"writing. meeting new people, spending time wi...",christianity,0.0,theism
4,24,single,f,straight,NaN,strictly anything,socially,NaN,graduated from college/university,white,...,i am always willing to try new foods and am no...,sports/my softball glove coffee. because nobod...,NaN,"in or out... drinking with friends, maybe a ba...",potential friends/lovers/people who come in co...,http://www.youtube.com/watch?v=4dxbwzuwsxk let...,"oh goodness. at the moment i have 4 jobs, so ...",christianity,2.0,theism


In [2]:
# get individuals whose seriousness_degree is 4.0 
serious_df = df[df.seriousness_degree == 4.0]

serious_df.columns = serious_df.columns.str.replace('[^\w\s]', '', regex=True)

# see the number of beliefs in serious_df
serious_df.cleaned_religion.value_counts()

cleaned_religion
atheism         554
christianity    543
other           508
agnosticism     311
catholicism      93
buddhism         66
judaism          20
hinduism         14
islam            11
Name: count, dtype: int64

In [3]:
# replace the space in the column name with underscore
serious_df.columns = serious_df.columns.str.replace(' ', '_')

# column names to lower case
serious_df.columns = serious_df.columns.str.lower()

# preview the data
serious_df.head()

,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,favorite_books_movies_show_music_and_food,the_six_things_i_could_never_do_without,i_spend_a_lot_of_time_thinking_about,on_a_typical_friday_night_i_am,the_most_private_thing_i_am_willing_to_admit,you_should_message_me_if,merged_profile,cleaned_religion,seriousness_degree,belief
0,22,single,m,straight,a little extra,strictly anything,socially,never,working on college/university,"asian, white",...,"books: absurdistan, the republic, of mice and ...",food. water. cell phone. shelter.,duality and humorous things,trying to find someone to hang out with. i am ...,i am new to california and looking for someone...,you want to be swept off your feet! you are ti...,about me: i would love to think that i was so...,agnosticism,4.0,agnosticism
6,28,seeing someone,m,straight,average,mostly anything,socially,never,graduated from college/university,white,...,books = yes. avid reader. moves = eternal suns...,"guitar - even if i don't play it all the time,...",a little bit of everything. but mostly social ...,hanging out with a small group of friends--sta...,i'm picky when it comes to dating. i know what...,"if you know who you are, who you want, where y...","i was born in wisconsin, grew up in iowa, and ...",christianity,4.0,theism
46,31,single,f,bisexual,average,strictly anything,socially,NaN,NaN,"middle eastern, other",...,"""wings of desire"" (german film by wim wenders)...",i can do without. i value: 1. my hands. 2. ot...,"i rarely think, it's over-rated ; )",the answer to each moment must be yes.,"i am an open book, i do not like secrets. if i...",you feel compelled.,i slept and dreamt that life was joy. i awoke ...,other,4.0,other
48,35,single,m,straight,athletic,mostly anything,socially,sometimes,graduated from space camp,"native american, white",...,NaN,"my camera, aviation, amazing food, touch, lear...","travel, food, photography, sensual fun, aviati...","often working a party. lights, sound, managing...",NaN,you can make me laugh or turn me on. teach me ...,"i'm an adventurer first, i take calculated ris...",agnosticism,4.0,agnosticism
59,29,single,m,straight,fit,mostly anything,socially,sometimes,graduated from college/university,white,...,-books: anything joseph campbell - osho - terr...,invalid question,the world,out.,no,you're curious.,"my names josh, and i create art for a living. ...",agnosticism,4.0,agnosticism


In [4]:
# filter out NA entries in 'you_should_message_me_if...'
serious_df = serious_df[serious_df.you_should_message_me_if.notna()]

# filter out string with less than 25 characters (super short and careless responses)
serious_df = serious_df[serious_df.you_should_message_me_if.str.len() > 30]

In [5]:
serious_df_male = serious_df[serious_df['sex'] == 'm']
serious_df_female = serious_df[serious_df['sex'] == 'f']

In [6]:
from numpy.random import choice

# List of religions to include, excluding 'other'
religions = ['atheism', 'christianity', 'agnosticism', 'catholicism', 'buddhism', 'judaism', 'hinduism', 'islam']

In [7]:
# Dictionary to hold the sampled individuals from each religion
sampled_individuals_male = {}

# Loop through each religion and sample individuals
for religion in religions:
    # Filter the dataset for the current religion
    religion_data = serious_df_male[serious_df_male['cleaned_religion'] == religion]
    
    # Number of individuals to sample: 15 or the size of the group if it has fewer than 15 members
    n_samples = min(50, len(religion_data))
    
    # Randomly sample individuals without replacement
    sampled_individuals_male[religion] = religion_data.sample(n=n_samples, replace=False)

# Combine all sampled individuals into a single DataFrame
male_sample = pd.concat(sampled_individuals_male.values())

male_sample

,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,favorite_books_movies_show_music_and_food,the_six_things_i_could_never_do_without,i_spend_a_lot_of_time_thinking_about,on_a_typical_friday_night_i_am,the_most_private_thing_i_am_willing_to_admit,you_should_message_me_if,merged_profile,cleaned_religion,seriousness_degree,belief
36900,33,single,m,straight,average,vegan,socially,NaN,NaN,"hispanic / latin, white",...,"magazines... xlr8r,fader,juxtapoz. movies...su...",headphones ...saturdays...the perfect hoodie.....,how to get my on-line record/entertainment com...,typically i'm making dinner with my roommates ...,i got nothing rite now...,everyday is an adventure. not a flake.not reli...,i'm generally a pretty laid back person when i...,atheism,4.0,atheism
8612,29,single,m,straight,athletic,NaN,not at all,never,graduated from college/university,NaN,...,"books: nabokov, updike, tolstoy, and proust, a...",irony. servants. the languid contemplation of ...,why this little news feed here sees fit to kee...,being put in a corner and told not to speak.,i once came to in the ice-filled bathtub of a ...,you've been a very bad girl and need a stern t...,why does man kill? he kills for food. and not ...,atheism,4.0,atheism
15295,20,single,m,straight,average,mostly anything,socially,sometimes,working on two-year college,white,...,"i really like black flag, my bloody valentine,...",food and watery type stuff,all kinds of stuff,"going to/playing a show, partying, keeping it ...",NaN,if you think we would get along.,all the obvious stuff. i'm a cook. playing in ...,atheism,4.0,atheism
34628,38,seeing someone,m,straight,NaN,NaN,rarely,never,NaN,white,...,books: no short answer possible - i moved boxe...,"1. my glasses - without them, i'm blind. 2. a...",new ideas. occasionally ones that i think migh...,NaN,well actually a few but one is that while i am...,you are a geek - which doesn't just mean techn...,my friends turn to me for restaurant recommend...,atheism,4.0,atheism
4115,30,single,m,straight,athletic,anything,socially,NaN,graduated from college/university,white,...,books: i am whittling through 100 classics at ...,"family, helmets, creative outlets, kitchen, fr...","earthquakes. i have yet to experience one, and...",trying out some place new or trying to cook so...,"i have been playing set! in my spare time, to ...","you are looking for a good meal, a good conver...",my new year's resolution this year was to step...,atheism,4.0,atheism
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,39,single,m,straight,athletic,strictly anything,NaN,NaN,graduated from space camp,white,...,"holy shit, it's data mining! fuck you, okc! m...",water. a nitrox atmosphere. cerebral cortex. m...,why riot grrls went extinct so quickly.,wishing i had someone as passionate and decade...,"my 92-year-old grandma better like you, becaus...",you've never attended: a) sacred heart cathedr...,"""essay?"" fat fucking chance. playwright. phot...",islam,4.0,theism
8143,23,single,m,straight,fit,mostly anything,socially,never,graduated from masters program,asian,...,john steinbeck is my fav writer. the grapes o...,"family allah, god, bhagwan or whatever you wan...",*what the heck i want to accomplish with my li...,hanging out with friends and drink as much as ...,nooooooooo i am not going to tell :),you are funny and love to make friends. :),what to tell about myself. :) i am an ipad dev...,islam,4.0,theism
29225,27,single,m,straight,overweight,mostly vegetarian,not at all,never,graduated from masters program,asian,...,books the dummy line enders game 3 mistakes o...,NaN,"about my family, friends and how can i make a ...",NaN,i got enrolled on this app :-),if you wanna have a good friend to have some g...,books the dummy line enders game 3 mistak...,islam,4.0,theism
21604,33,single,m,straight,thin,mostly anything,socially,never,graduated from masters program,middle eastern,...,i'm a typical guy when it 

In [8]:
# check count 
male_sample.cleaned_religion.value_counts()

cleaned_religion
atheism         50
christianity    50
agnosticism     50
catholicism     28
buddhism        21
judaism          5
hinduism         5
islam            5
Name: count, dtype: int64

In [9]:
# Dictionary to hold the sampled individuals from each religion
sampled_individuals_female = {}

# Loop through each religion and sample individuals
for religion in religions:
    # Filter the dataset for the current religion
    religion_data = serious_df_female[serious_df_female['cleaned_religion'] == religion]
    
    # Number of individuals to sample: 15 or the size of the group if it has fewer than 15 members
    n_samples = min(50, len(religion_data))
    
    # Randomly sample individuals without replacement
    sampled_individuals_female[religion] = religion_data.sample(n=n_samples, replace=False)

# Combine all sampled individuals into a single DataFrame
female_sample = pd.concat(sampled_individuals_female.values())

female_sample

,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,favorite_books_movies_show_music_and_food,the_six_things_i_could_never_do_without,i_spend_a_lot_of_time_thinking_about,on_a_typical_friday_night_i_am,the_most_private_thing_i_am_willing_to_admit,you_should_message_me_if,merged_profile,cleaned_religion,seriousness_degree,belief
38292,38,single,f,straight,a little extra,NaN,socially,NaN,graduated from college/university,white,...,my favorite books are by david sedaris and aug...,my two passports friends wine cheese music europe,where and when my next vacation will be.,going out to dinner and drinks with friends or...,there's really nothing private about me.,you think you can make me laugh!,"i grew up mostly in the bay area, lived in san...",atheism,4.0,atheism
2253,27,single,f,bisexual,thin,anything,not at all,often,graduated from college/university,white,...,"""favorite"" is not really a category i collect ...",1. books -- there's not enough time that can b...,how to make the world a better place. how to g...,doing something boring like watching tv on my ...,NaN,...you're open to being open! i've tried monog...,i'm currently working to build my petsitting ...,atheism,4.0,atheism
7839,25,single,f,bisexual,thin,strictly vegetarian,socially,never,graduated from college/university,white,...,"yes, please.",traumatic middle school memories bed being abl...,"why the ocean is salty. why ponds, streams, ri...","partying with the hipsters, but really just sp...","i'm really, really bad at math.","you speak sarcasm, appreciate irony, but don't...","oh hey there, internet person. i hope you find...",atheism,4.0,atheism
35407,20,single,f,bisexual,average,mostly vegetarian,often,often,dropped out of two-year college,white,...,"books: most chuck palahniuk books, the little ...",kittens bloody mary's sandwiches orgasms math ...,how to reach inner fulfillment and total happi...,"greeting the weekend with open arms, booze, we...",my dreams always suck,you enjoy a good fleshless sandwich. and/or yo...,"i am a french native, california transplant. w...",atheism,4.0,atheism
29066,31,single,f,straight,used up,strictly vegetarian,socially,sometimes,graduated from law school,white,...,"books: kafka on the shore, geek love, the wind...",1. animal friends 2. music 3. a camera 4. srir...,bigfoot,trying to prevent my cat from eating my prized...,i have a baby-bird phobia.,you like drinking fancy beers with girls who c...,"potty-mouthed and cat-obsessed, what more coul...",atheism,4.0,atheism
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19125,20,single,f,straight,overweight,mostly halal,not at all,often,working on high school,"black, pacific islander",...,the brothers solomon. the bibel. hansen. siste...,twin 1 twin 2 bun peanut butter polka-dot tigh...,daaaad. whare r yoo.,being a twin.,we have different moms.,you know our dad/ you are a dad/ you are our d...,two twin sisters. us: http://www.youtube.com/...,hinduism,4.0,theism
29846,19,single,f,straight,skinny,NaN,not at all,never,working on high school,indian,...,all types of movies especially comedy movies. ...,my cellphone my younger sister nikita and my e...,getting that right man in my life who can love...,at home or visit my families around sf bay are...,i will get married at age 21 it will be arrang...,"you like my pictures, and if you are a kind an...","i am avishta, 18 years old, live in redwood ci...",hinduism,4.0,theism
13376,44,single,f,straight,curvy,mostly vegetarian,rarely,never,working on two-year college,pacific islander,...,"favorite books-the path to mother, women who r...",love in all it's many forms (children/friends/...,everything & nothing taking care of this momen...,online watching a live webinar for my ayurvedi...,to a fault...don't ask me a question unless yo...,i'm wanting to accept my partner for who they ...,i give diversity a whole new definition. i'm s...,hinduism,4.0,theism
24253,39,single,f,straight,c

In [10]:
# check count 
female_sample.cleaned_religion.value_counts()

cleaned_religion
atheism         50
christianity    50
agnosticism     50
catholicism     27
buddhism        26
judaism          9
hinduism         5
islam            1
Name: count, dtype: int64

## Apply embedding to the dating profiles

In [11]:
api_key = os.getenv('OPENAI_API_KEY')

In [12]:
from openai import OpenAI
client = OpenAI()

def get_embedding(text, model="text-embedding-3-large"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

In [13]:
# apply the function to the final_sample
female_sample['embedding'] = female_sample['you_should_message_me_if'].apply(get_embedding)

male_sample['embedding'] = male_sample['you_should_message_me_if'].apply(get_embedding)

In [15]:
# combine female_sample and male_sample into a single dataframe
final_sample = pd.concat([female_sample, male_sample])

In [16]:
#preview the final_sample
final_sample.head()

,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,the_six_things_i_could_never_do_without,i_spend_a_lot_of_time_thinking_about,on_a_typical_friday_night_i_am,the_most_private_thing_i_am_willing_to_admit,you_should_message_me_if,merged_profile,cleaned_religion,seriousness_degree,belief,embedding
38292,38,single,f,straight,a little extra,NaN,socially,NaN,graduated from college/university,white,...,my two passports friends wine cheese music europe,where and when my next vacation will be.,going out to dinner and drinks with friends or...,there's really nothing private about me.,you think you can make me laugh!,"i grew up mostly in the bay area, lived in san...",atheism,4.0,atheism,"[0.005271855276077986, -0.0013254006626084447,..."
2253,27,single,f,bisexual,thin,anything,not at all,often,graduated from college/university,white,...,1. books -- there's not enough time that can b...,how to make the world a better place. how to g...,doing something boring like watching tv on my ...,NaN,...you're open to being open! i've tried monog...,i'm currently working to build my petsitting ...,atheism,4.0,atheism,"[-0.031448844820261, -0.02344522811472416, -0...."
7839,25,single,f,bisexual,thin,strictly vegetarian,socially,never,graduated from college/university,white,...,traumatic middle school memories bed being abl...,"why the ocean is salty. why ponds, streams, ri...","partying with the hipsters, but really just sp...","i'm really, really bad at math.","you speak sarcasm, appreciate irony, but don't...","oh hey there, internet person. i hope you find...",atheism,4.0,atheism,"[-0.023152481764554977, -0.005695262923836708,..."
35407,20,single,f,bisexual,average,mostly vegetarian,often,often,dropped out of two-year college,white,...,kittens bloody mary's sandwiches orgasms math ...,how to reach inner fulfillment and total happi...,"greeting the weekend with open arms, booze, we...",my dreams always suck,you enjoy a good fleshless sandwich. and/or yo...,"i am a french native, california transplant. w...",atheism,4.0,atheism,"[-0.010571823455393314, -0.035015273839235306,..."
29066,31,single,f,straight,used up,strictly vegetarian,socially,sometimes,graduated from law school,white,...,1. animal friends 2. music 3. a camera 4. srir...,bigfoot,trying to prevent my cat from eating my prized...,i have a baby-bird phobia.,you like drinking fancy beers with girls who c...,"potty-mouthed and cat-obsessed, what more coul...",atheism,4.0,atheism,"[0.022974137216806412, -0.011390862055122852, ..."


In [17]:
# save the final_sample to a csv file
final_sample.to_csv('sample_with_description.csv', index=False)

## Embed stereotypical descriptions

In [18]:
dating_profiles = {
    "Housewife": "Hello! I'm a devoted homemaker, cherishing every moment spent nurturing my family in a cozy, well-kept home. My heart is in transforming everyday tasks into acts of love, whether it's baking family recipes, tending to our garden, or handcrafting decorations that turn our house into a home. I'm here to connect with friends who revel in the art of homemaking or to find a partner who sees the beauty in a traditional, loving household. Let's exchange our favorite family traditions, craft ideas, and the little joys that make a house a home!",
    
    "One-Night Stand Seeker": "Hey! I'm all about seizing the day and embracing the excitement of fleeting connections. If your idea of a good time involves exhilarating, commitment-free adventures that last just for the night, we're on the same page. I value honesty, respect, and the thrill of a spontaneous rendezvous. Let's make our encounter unforgettable, with the promise of no expectations when the sun comes up!",
    
    "DINK": "Greetings! I'm deeply invested in my career and savor the pleasures of a child-free existence. Luxurious getaways, gourmet dining, and impromptu adventures define my free time. I'm on the lookout for someone who's equally ambitious, enjoys a certain level of sophistication, and cherishes the liberties our lifestyle affords. If you're drawn to a life of exploration and shared success, let's chart our course to extraordinary experiences!",
    
    "Materialistic": "Salutations! For me, life's greatest joys are found in its luxuries—designer brands, exclusive events, and gourmet dining. I'm searching for a partner who not only understands the allure of high-end living but also strives to embody it. If you're someone who appreciates the finer things and views life as a collection of exquisite experiences, we might just be a perfect match. Let's indulge in the opulence life has to offer, together.",
    
    "Family-Oriented": "Hello! Envisioning a home brimming with laughter, love, and the bustling energy of children is my ultimate dream. I hold family values dear and am eager to create a life filled with meaningful traditions and shared moments. If you're longing for a future rich in family gatherings, heartfelt connections, and the delightful chaos of a big family, I'm looking for you. Let's build a foundation of love that grows with every new addition to our family.",
    
    "Virgin": "Hi! Stepping into the dating scene with eyes wide open, I'm searching for someone who treasures emotional depth and genuine connections. My ideal relationship evolves slowly, grounded in mutual respect and shared discoveries. If you're patient, value heartfelt conversations, and believe that the best connections are more than physical, I'm eager to explore what we might find together.",
    
    "Super Religious": "Blessings! My life is a testament to my unwavering faith, with every action guided by divine love and teachings. I find joy in communal worship, charitable acts, and deep theological study. I seek a partner for whom faith is either a cornerstone or a respected aspect of life, with whom I can share a journey of spiritual growth, service, and a life that mirrors our highest ideals. If you're called to a life of faith and fellowship, let's walk this path hand in hand.",
    
    "Adventure Seeker": "Hey there! With a heart that races for the next adrenaline rush, I'm always on the lookout for the next heart-pounding adventure. Skydiving, bungee jumping, or wild surfing—anything that gets my blood pumping is on my list. I'm in search of a fearless partner who's ready to embrace the extreme and dive headfirst into life's greatest adventures. If you're eager for thrills and spills, let's take the plunge together!",
    
    "Tech Geek": "Greetings! Immersed in the tech world, my life orbits around the latest innovations, from groundbreaking software to cutting-edge gadgets. I relish weekends spent coding, gaming in virtual realms, or debating the future of AI. I thrive on change and am constantly driven by the quest for knowledge. I'm looking for a partner who's either a fellow tech aficionado or keen to delve into the digital universe with me. Let's sync our worlds and maybe create something revolutionary.",
    
    "Eco-Conscious Individual": "Hi! Living in harmony with our planet is my daily mantra, from advocating for green policies to nurturing my own sustainable garden. I'm committed to making a positive impact and seek a partner who shares this passion for conservation and environmental stewardship. If you're drawn to a life of purpose, sustainability, and outdoor adventures, let's join forces to make a difference, one eco-friendly step at a time.",
    
    "Fitness Fanatic": "Hello! For me, fitness is more than a routine—it's the essence of my lifestyle. I'm dedicated to pushing my limits, whether it's in the gym or on the running track. I'm on the hunt for a partner who shares this dedication to health and vitality. If you're inspired by setting goals and smashing them, and you see the value in a partner who motivates you to achieve your best, let's team up and keep each other driven.",
    
    "Artist": "Hello! As an artist, my world is a blend of colors, emotions, and creativity. I see art in everything and express my soul through my work. My passion lies in capturing the essence of moments and turning them into visual narratives. I'm seeking someone who appreciates art's power to move and transform us, who sees the world through a lens of wonder and imagination. Whether you're an artist or an art lover, let's connect and share our visions, creating beauty in the spaces between.",
    
    "Pet Lover": "Hello! My life is joyfully intertwined with my pets, who are as much a part of my family as anyone could be. I'm looking for a kindred spirit who understands that pets are not just animals, but cherished family members. Whether it's sharing stories about our furry friends or planning pet-friendly adventures, I'm looking for a partner who believes that love extends to all creatures, great and small.",
    
    "Culinary Enthusiast": "Hi! The world of flavors and culinary arts is where I find my bliss, from experimenting with exotic recipes to savoring the ambiance of quaint cafes. I'm in search of a partner who delights in the sensory pleasures of food and is eager to explore the gastronomic world with me. Whether we're cooking up a storm at home or embarking on a quest for the perfect dish, let's savor the journey together.",
    
    "Finance Bro": "Hey! In the fast-paced arena of finance, I thrive on the thrill of the deal and the buzz of the trading floor. My downtime is about enjoying the rewards, from exclusive clubs to luxury escapes. I'm looking for someone who appreciates the drive it takes to succeed and enjoys the finer things in life, someone who's not just a partner but a confidante in both business and pleasure. Let's share the highs and lows of the financial world and build our empire together.",
    
    "Startup CEO": "Hello, visionary! Leading a startup is more than a job; it's my calling, blending innovation with endless hustle. My life is a tapestry of pitch sessions, creative brainstorming, and the relentless pursuit of breakthroughs. I'm seeking someone who's either navigating their own entrepreneurial journey or has a zest for the startup ecosystem. If you're about dreaming big, embracing challenges, and making an impact, let's join forces and maybe set the world alight.",
    
    "Party Animal": "Hey! If life is a party, I'm living it 24/7. From electrifying club nights to epic music festivals, I'm all about the energy and pulse of the nightlife. I'm on the lookout for someone who matches my enthusiasm for fun, dance, and unforgettable nights out. If you're someone who lives for the moment and loves the spotlight as much as I do, let's make every night legendary.",
    
    "Adult Entertainment Professional": "Hello! In the world of adult entertainment, I'm dedicated to my craft, exploring the spectrum of human desire with artistry and openness. Beyond the camera, I'm just like anyone else, with hobbies and interests that range from the great outdoors to culinary arts. I value honesty, respect, and open communication in all my interactions. I'm here to connect with those who see beyond labels, eager for genuine connections and shared interests.",
    
    "Vegan": "Hi! My vegan journey is a reflection of my ethics, health, and environmental consciousness. It's about making compassionate choices that align with my values. I'm passionate about creating delectable vegan dishes and exploring sustainable living in all its forms. I'm here to meet someone who either shares this lifestyle or is open to exploring it together. Whether we're trying new recipes or enjoying nature's beauty, let's inspire each other towards a more compassionate, sustainable world.",
    
    "James Evans": "I am the Director of the Knowledge Lab, a Fellow in the Computation Institute, and the Co-Director for the Masters in Computational Social Science Program. In addition to my leadership roles, I hold the position of Max Palevsky Professor in Sociology. My research delves into the collective system of thinking and knowing, exploring everything from the distribution of attention and intuition to the origin of ideas and shared habits of reasoning, including processes of agreement and dispute, as well as the accumulation of certainty and doubt. I am particularly fascinated by innovation—how new ideas and practices take shape and the influence of social and technical institutions like the Internet, markets, and collaborations on collective cognition and discovery. My work primarily focuses on the collective system of thinking and knowing, examining the distribution of attention and intuition, the origin of ideas, and shared habits of reasoning, alongside the dynamics of agreement and dispute, and the nuanced texture of human understanding, including its novelty, ambiguity, and topology. Innovation captures my interest, especially in how new ideas and practices emerge and the role that social and technical institutions play in the collective process of cognition and discovery. Much of my research centers on modern science and technology, but I also delve into other knowledge domains such as news, law, religion, gossip, hunches, and historical modes of thinking and knowing. I am committed to creating novel observatories for human understanding and action, utilizing crowd sourcing, information extraction from text and images, and distributed sensors like RFID tags and cell phones. My approach includes using machine learning, generative modeling, and social and semantic network representations to explore knowledge processes, enhance interpretive and field methods, and develop alternatives to current discovery regimes. My research receives support from the National Science Foundation, the National Institutes of Health, the Templeton Foundation, and other sources. It has been published in prestigious journals such as Science, American Journal of Sociology, Social Studies of Science, and Administrative Science Quarterly, among others. My work has also been featured in major media outlets like Nature, The Economist, Atlantic Monthly, Wired, NPR, BBC, El País, CNN, and many more. At the University of Chicago, I lead the Knowledge Lab, offering collaborative, granting, and employment opportunities, along with ongoing seminars. I sponsor the Computational Social Science workshop alongside John Brehm, and the Knowledge-Value workshop with John Kelly, and co-organize the Rational Choice workshop with Gary Becker, Richard Posner, and Glen Weyl. I teach courses on the history of modern science, science studies, computational content analysis, and Internet and Society. Before joining Chicago, I earned my doctorate in sociology from Stanford University, worked as a research associate in the Negotiation, Organizations, and Markets group at Harvard Business School, founded a private high school focused on project-based arts education, and completed a BA in Anthropology and Economics at Brigham Young University."
}

In [19]:
# turn dating_profiles into a dataframe
profile_df = pd.DataFrame({'profile': list(dating_profiles.keys()), 'description': list(dating_profiles.values())})

In [20]:
# preview the profile_df
profile_df.head(3)

,profile,description
0,Housewife,"Hello! I'm a devoted homemaker, cherishing eve..."
1,One-Night Stand Seeker,Hey! I'm all about seizing the day and embraci...
2,DINK,Greetings! I'm deeply invested in my career an...


In [21]:
# apply the function to the profile_df
profile_df['embedding'] = profile_df['description'].apply(get_embedding)

In [22]:
# save the trait_df to a csv file
profile_df.to_csv('description_embedding_df.csv', index=False)